In [6]:
%pylab inline
import scipy.signal
import time


from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import kneighbors_graph
from sklearn.cluster import DBSCAN
import scipy.interpolate as inter
import matplotlib.pyplot as plt
import glob
import bz2


Populating the interactive namespace from numpy and matplotlib


/home/bye3/.local/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['power']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


 # All the Definitions Goes Here

In [14]:
# ALL DEFINITIONS

def vertical_len(single_freq_iono, y):
    inc = 0;
    while single_freq_iono[y + inc] > 1 and (y + inc) < len(single_freq_iono):
        inc += 1;
    return inc

def neighbor_len(single_freq_iono, y):
    inc = 0;
    while single_freq_iono[y - inc] > 1 and (y - inc) > 0:
        inc += 1;
    total = inc + vertical_len(single_freq_iono, y)
    if (total > 1):
        total -=1
    return total


def shrink(data_array_1):
    y_dim, x_dim = data_array_1.shape
    data_array = data_array_1.copy()
    for i in range(0, x_dim):
        single_freq_iono = data_array[:, i]
        for j in range(0, y_dim - 2):
            if (single_freq_iono[j] > 1 and single_freq_iono[j + 1] > 1):
                if i != x_dim - 1:
                    neighbor_len_right = neighbor_len(data_array[:, i + 1], j)
                    print(neighbor_len_right)
                    if (neighbor_len_right >= 2):
                        inc = vertical_len(single_freq_iono, j)
                        print(inc)
                        for k in range(j, j + inc):
                            if k != (np.floor((j + j + inc)/2)):
                                print(k)
                                single_freq_iono[k] = 0
                                print(single_freq_iono)
        data_array[:, i] = single_freq_iono
    return data_array

def neighbor_exist(single_freq_iono, y):
    if ((y - 1) > 0 and (single_freq_iono[y] and single_freq_iono[y - 1])) or ((y + 1) < len(single_freq_iono) and (single_freq_iono[y] and single_freq_iono[y + 1])):
        return 1
    return 0



def neighbor_check(data_array_1):
    y_dim, x_dim = data_array_1.shape
    data_array = data_array_1.copy()
    for i in range(1, y_dim - 1):
        for j in range(1, x_dim - 1):
            if not (data_array[i - 1, j - 1] or data_array[i - 1, j] or data_array[i - 1, j + 1] 
                    or data_array[i, j - 1] or data_array[i, j + 1]
                    or data_array[i + 1, j - 1] or data_array[i + 1, j] or data_array[i + 1, j + 1]):
                    data_array[i, j] = 0
    return data_array


def find_min_xy(arr2d):
    # This function finds the minimun x and y coordinates of a cluster
    min_x = np.inf
    min_y = np.inf
    for i in range(0, len(arr2d)):
        if arr2d[i, 0] < min_x:
            min_x = arr2d[i, 0]
        if arr2d[i, 1] < min_y:
            min_y = arr2d[i, 1]
    return min_x, min_y

def find_max_xy(arr2d):
    # This function finds the maximun x and y coordinates of a cluster
    max_x = 0
    max_y = 0
    for i in range(0, len(arr2d)):
        if arr2d[i, 0] > max_x:
            max_x = arr2d[i, 0]
        if arr2d[i, 1] > max_y:
            max_y = arr2d[i, 1]
    return max_x, max_y

def find_max_area(arr2d):
    # This function finds the maximun area using min and max coords.
    return (find_max_xy(arr2d)[0] - find_min_xy(arr2d)[0])*(find_max_xy(arr2d)[1] - find_min_xy(arr2d)[1])


def group_labels(X_image, labels, n_clusters):
    # This function groups every cluster together.
    # Input X_image is a 2d array that are coordinate pairs;
    # Input labels are a 1d array specify the cluster label for each coordinate pairs.
    # Output group is a dictionary that has the format: {label: list of pairs}
    current_label = 0;
    group = {};
    for i in range(0, n_clusters ):
        group[i] = []

    for j in range(0, len(labels)):
        group[labels[j]].append(X_image[j])
    return group

def find_cluster(X_image, labels, n_clusters):
    group = group_labels(X_image, labels, n_clusters)
    group_want = 0
    max_area = 0
    for i in range(0, n_clusters):
        if find_max_area(array(group[i])) > max_area:
            max_area = find_max_area(array(group[i]))
            group_want = i
    return group_want

def coor_pair_to_mesh_array(arr2d):
    max_x, max_y = find_max_xy(arr2d)
    mesh_array = np.zeros(np.array([max_y + 1, max_x + 1]).astype(int))
    for i in range(0, len(arr2d)):
        coord = arr2d[i].astype(int)
        mesh_array[coord[1], coord[0]] = 10
    return mesh_array

def avg_y(arr2d):
    return np.mean(arr2d[:, 1])

def max_num_group_and_len(sub_cluster):
    current_len = 0
    current_group = 0
    for i in range(0, len(sub_cluster)):
        if len(sub_cluster[i]) > current_len:
            current_len = len(sub_cluster[i])
            current_group = i
    return current_group, current_len

def find_nearst_cluster(predicted, sub_cluster_avg):
    modified = abs(np.array(sub_cluster_avg) - predicted)
    compare = np.inf
    ind = -1
    for i in range(0, len(modified)):
        if modified[i] < 40 and modified[i] < compare:
            compare = modified[i]
            ind = i
    if ind == -1:
        return ind
    return ind, sub_cluster_avg[ind]

def vertical_noise_substraction(data):
    #data_new = np.log10(data)
    data_new = data.copy()
    data_new[0:7, :] = 0
    data_new[:, 0:150] = 0
    data_new[:, -10:-1] = 0
    for trial in range(0, 3):
        for j in range(0, len(freqs)):
            single_freq_iono = array(data_new[:,j])
            avg_power = single_freq_iono.sum()*1.0/(len(single_freq_iono))
            noise_sum = 0
            max_power = np.max(single_freq_iono)
            median_power = np.median(single_freq_iono)
            #'''
            for i in range(0, len(single_freq_iono)):
                if (single_freq_iono[i] < 0.5*max_power):
                    noise_sum += single_freq_iono[i]
            noise = noise_sum/len(single_freq_iono)
            for i in range(0, len(single_freq_iono)):
                if (single_freq_iono[i] - noise < 1):
                    single_freq_iono[i] = 1
                else:
                    single_freq_iono[i] = single_freq_iono[i] - noise

            data_new[:,j] = single_freq_iono*1
    return data_new

# find nnz in an array
def find_nnz(data):
    nnz = 0
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            if data[i, j] != 0:
                nnz += 1
    return nnz

def mean_clear(data_after_substraction):
    data_processed = data_after_substraction.copy()
    nnz = find_nnz(data_processed)
    array_mean = np.sum(data_processed)/nnz
    for i in range(data_processed.shape[0]):
        for j in range(data_processed.shape[1]):
            if data_processed[i, j] > array_mean*0.85:
                data_processed[i, j] = 10
            else:
                data_processed[i, j] = 0
    return data_processed

def whole_clean_procedure():
    # whole procedure, below this cell is individual step
    iono_num = 0

    data_processed = mean_clear(vertical_noise_substraction(ionograms[iono_num,:,:]))
    nnz = find_nnz(data_processed)
    x_image = np.zeros(nnz)
    y_image = np.zeros(nnz)
    ind = 0
    for i in range(data_processed.shape[0]):
        for j in range(data_processed.shape[1]):
            if data_processed[i, j] > 1:
                x_image[ind] = i;
                y_image[ind] = j;
                ind += 1

    X_image = np.concatenate((array([y_image]), array([x_image])))
    X_image = X_image.T

    n_clusters = 75
    plt.figure(figsize=(20, 10))
    model = AgglomerativeClustering(linkage="single",
                                    connectivity=None,
                                    n_clusters=n_clusters)
    model.fit(X_image)

    group_want = find_cluster(X_image, model.labels_, n_clusters)
    X_image_want = array(group_labels(X_image, model.labels_, n_clusters)[group_want])

    x_group = X_image_want[:, 0]
    y_group = X_image_want[:, 1]

    data_final = np.zeros(data_processed.shape)
    for i in range(0, len(X_image_want)):
        row = int(X_image_want[i, 0]);
        col = int(X_image_want[i, 1]);
        data_final[col, row] = 10

    fig = figure(figsize=(14, 7))
    im = pcolormesh(freqs,hts,data_final)
    im.set_clim(0.2,9.8)
    im.axes.set_xlim(2,12); im.axes.set_ylim(50,750); 
    plt.xticks(np.linspace(freqs.min(), freqs.max(), 20))
    im.axes.set_title(filename + "_cleaned")
    
    # show and save
    #show(im)
    fig.savefig(folderout+'/'+filename[6:6+13]+'_OXgrams.png')   # png format
    fig.clf()
    

In [15]:
# Read File
# data type definition	
iq_t = dtype([('I','>i2'),('Q','>i2')])	
# header type definitions
sct_t = dtype([('magic','<u4'),('sounding_table_size','<u4'), ('pulse_table_size','<u4'), ('raw_data_size','<u4'), 
    ('struct_version','<f4'),('start_year','<u4'), ('start_daynumber','<u4'), ('start_month','<u4'), ('start_day','<u4'), 
    ('start_hour','<u4'), ('start_minute','<u4'), ('start_second','<u4'), ('start_epoch','<u4'), 
    ('readme','128<int8'), ('decimation_method','<u4'), ('decimation_threshold','<f4'), ('user','128<int8')])
station_t = dtype([('file_id','64<int8'), 
    ('ursi','8<int8'), ('rx_name','32<int8'), ('rx_latitude','<f4'), ('rx_longitude','<f4'), ('rx_altitude','<f4'), 
    ('rx_count','<u4'), ('rx_antenna_type','(32,32)<int8'), ('rx_position','(32,3)<f4'), ('rx_direction','(32,3)<f4'), 
    ('rx_height','32<f4'), ('rx_cable_length','32<f4'), ('frontend_atten','<f4'), ('tx_name','32<int8'), ('tx_latitude','<f4'), 
    ('tx_longitude','<f4'), ('tx_altitude','<f4'), ('tx_antenna_type','32<int8'), ('tx_vector','3<f4'), ('tx_height','<f4'), 
    ('tx_cable_length','<f4'), ('drive_band_count','<u4'), ('drive_band_bounds','(64,2)<f4'), ('drive_band_atten','64<f4'), 
    ('rf_control','<u4'), ('ref_type','32<int8'), ('clock_type','32<int8'), ('user','128<int8')])	
timing_t = dtype([ ('file_id','64<int8'), 
    ('pri','<f4'), ('pri_count','<u4'), ('ionogram_count','<u4'), ('holdoff','<f4'), ('range_gate_offset','<f4'), 
    ('gate_count','<u4'), ('gate_start','<f4'), ('gate_end','<f4'), ('gate_step','<f4'), ('data_start','<f4'), ('data_width','<f4'), 
    ('data_baud_count','<u4'), ('data_wave_file','64<int8'), ('data_baud','(1024,2)<f4'), ('data_pairs','<u4'), ('cal_start','<f4'), 
    ('cal_width','<f4'), ('cal_baud_count','<u4'), ('cal_wave_file','64<int8'), ('cal_baud','(1024,2)<f4'), ('cal_pairs','<u4'), 
    ('user','128<int8')])
frequency_t = dtype([('file_id','64<int8'), 
    ('base_start','<f4'), ('base_end','<f4'), ('base_steps','<u4'), ('tune_type','<u4'), 
    ('base_table','8192<f4'), ('linear_step','<f4'), ('log_step','<f4'), ('freq_table_id','64<int8'), ('tune_steps','<u4'), 
    ('pulse_count','<u4'), ('pulse_pattern','256<u4'), ('pulse_offset','<f4'), ('ramp_steps','<u4'), 
    ('ramp_repeats','<u4'), ('drive_table','8192<f4'), ('user','128<int8')])
receiver_t = dtype([('file_id','64<int8'), 
    ('rx_chan','<u4'), ('rx_map','16<u4'), ('word_format','<u4'), ('cic2_dec','<u4'), ('cic2_interp','<u4'), 
    ('cic2_scale','<u4'), ('cic5_dec','<u4'), ('cic5_scale','<u4'), ('rcf_type','32<int8'), ('rcf_dec','<u4'), ('rcf_taps','<u4'), 
    ('coefficients','160<u4'), ('analog_delay','<f4'), ('user','128<int8')])
exciter_t = dtype([('file_id','64<int8'), 
    ('cic_scale','<u4'), ('cic2_dec','<u4'), ('cic2_interp','<u4'), ('cic5_interp','<u4'), 
    ('rcf_type','32<int8'), ('rcf_taps','<u4'), ('rcf_taps_phase','<u4'), ('coefficients','256<u4'), ('analog_delay','<f4'), 
    ('user','128<int8')])
monitor_t = dtype([
    ('balun_currents','8<u4'), ('balun_status','8<u4'), ('front_end_status','8<u4'), ('receiver_status','8<u4'), 
    ('exciter_status','2<u4'), ('user','512<int8')])
pct_t = dtype([
    ('record_id','<u4'), ('pri_ut','<f8'), ('pri_time_offset','<f8'), ('base_id','<u4'), ('pulse_id','<u4'), ('step_id','<u4'), 
    ('repeat_id','<u4'), ('loop_id','<u4'), ('frequency','<f4'), ('nco_tune_word','<u4'), ('drive_attenuation','<f4'), 
    ('pa_flags','<u4'), ('pa_forward_power','<f4'), ('pa_reflected_power','<f4'), ('pa_vswr','<f4'), ('pa_temperature','<f4'), 
    ('proc_range_count','<u4'), ('proc_noise_level','<f4'), ('user','64<int8')])




folders = ["014"]
for folder_num in folders:
    foldername = '/raid/VIPIR/vipir2015/%s/raw' %folder_num
    #filesname='*2018228*RIQ.bz2' # format JM91J_dddhhmmss.RIQ.bz2 
    #filesname='*2018228*RIQ' # format JM91J_dddhhmmss.RIQ
    #adjust regexp above to choose ionosondes to plot
    #filesname='*.RIQ.bz2'
    filesname='*.RIQ'
    datafiles = glob.glob1(foldername+'/',filesname) 
    datafiles.sort()

    rcParams.update({'xtick.labelsize':10,'ytick.labelsize':10})
    pngno=0

    for filename in datafiles[770:]:
        fullname = foldername+'/'+filename
        print(fullname)
        fid=open(fullname,'rb')  # opening *.RIQ files
        #fid=bz2.BZ2File(fullname,'rb') # opening *.RIQ.bz2 files

        print(fid)

    # read common headers
    # read with fromfile *.RIQ files: sct = fromfile(fid,sct_t,1) 
        sct = fromfile(fid,sct_t,1)
        station = fromfile(fid,station_t,1)
        timing = fromfile(fid,timing_t,1)
        frequency = fromfile(fid,frequency_t,1)
        receiver = fromfile(fid,receiver_t,1)
        exciter = fromfile(fid,exciter_t,1)
        monitor = fromfile(fid,monitor_t,1)
        '''
    # read with frombuffer *.RIQ.bz2 files:  
        sct = frombuffer(fid.read(sct_t.itemsize),sct_t,1)   
        station = frombuffer(fid.read(station_t.itemsize),station_t,1)
        timing = frombuffer(fid.read(timing_t.itemsize),timing_t,1)
        frequency = frombuffer(fid.read(frequency_t.itemsize),frequency_t,1)
        receiver = frombuffer(fid.read(receiver_t.itemsize),receiver_t,1)
        exciter = frombuffer(fid.read(exciter_t.itemsize),exciter_t,1)
        monitor = frombuffer(fid.read(monitor_t.itemsize),monitor_t,1)
        '''
    # extract useful header info
        rx_chan = receiver['rx_chan'][0]   # 8
        base_steps = frequency['base_steps'][0]   # variable
        pulse_count = frequency['pulse_count'][0]   # pulses per base_step
        ramp_repeats = frequency['ramp_repeats'][0]   # 
        ionogram_count = timing['ionogram_count'][0]
        gate_count=timing['gate_count'][0]   # hts sampled
        gate_start = timing['gate_start'][0]   # in us
        gate_step = timing['gate_step'][0]   # in us
        range_gate_offset = timing['range_gate_offset'][0]   # to be subtracted from gate_start 
        h0 = (gate_start-range_gate_offset)*0.15   # in km 
        dh = gate_step*0.15
        hlast = h0+dh*gate_count
        hts = arange(h0,hlast,dh)   # define height grid
        freqs = zeros(base_steps)   # initiate freq grid
        ionograms = zeros([rx_chan,gate_count,base_steps])   # initiate ionogram buffer
        tx_count = base_steps*ramp_repeats*pulse_count*ionogram_count   # total number of tx'ed pulses per ionogram

        for i in range(tx_count):   # frequency sweep loop
            pct = frombuffer(fid.read(pct_t.itemsize),pct_t,1)   # read frequency header
            base_id = pct['base_id'][0]
            pulse_id = pct['pulse_id'][0]
            step_id = pct['step_id'][0]
            repeat_id = pct['repeat_id'][0]
            iq = frombuffer(fid.read(iq_t.itemsize*rx_chan*gate_count),iq_t,rx_chan*gate_count)   # read raw I and Q returns per pulse
            volt = iq['I']+1j*iq['Q']   # form complex voltage I+jQ profile
            volt = volt.reshape(gate_count,rx_chan)
            power = abs(volt)**2

            voltOX=volt
            voltOX[:,0]=volt[:,0]+volt[:,1]
            voltOX[:,1]=volt[:,0]-volt[:,1]
            powerOX = abs(voltOX)**2   # square detection

            if i%20>3:
                ionograms[0,:,base_id-1] += power[:,0]   # accumulated ionograms
                ionograms[1,:,base_id-1] += power[:,1]
                ionograms[2,:,base_id-1] += powerOX[:,0]   # accumulated ionograms
                ionograms[3,:,base_id-1] += powerOX[:,1]

            #ionograms[2,:,base_id-1] += power[:,2]
            #ionograms[3,:,base_id-1] += power[:,3]
            #ionograms[4,:,base_id-1] += power[:,4]
            #ionograms[5,:,base_id-1] += power[:,5]
            #ionograms[6,:,base_id-1] += power[:,6]
            #ionograms[7,:,base_id-1] += power[:,7]
            freqs[base_id-1] = pct['frequency'][0]   # fill frequency grid

        freqs = freqs/1000.
        fid.close()   # vipir raw data file closed 


        #for i in range(size(freqs)):
        #    ionograms[1,i,:]=ionograms[1,i,:]-ionograms[1,i,0]

        '''
        ##### TEMP #####
        # Used to generate single image
        fig = figure(figsize=(14, 7))
        im = pcolormesh(freqs,hts,log10(ionograms[0,:,:]))
        im.set_clim(0.2,9.8)
        im.axes.set_xlim(2,12); im.axes.set_ylim(50,750); 
        plt.xticks(np.linspace(freqs.min(), freqs.max(), 20))
        im.axes.set_title(filename); im.axes.title.set_fontsize(10.0)
        colorbar()
        ################
        '''
        
        
        fig = figure(figsize=(14, 7))
        subplot(2,2,1)
        im = pcolormesh(freqs,hts,log10(ionograms[0,:,:]))
        im.set_clim(0.2,9.8)
        im.axes.set_xlim(2,12); im.axes.set_ylim(50,750); 
        im.axes.set_title(filename); im.axes.title.set_fontsize(10.0)
        colorbar()

        subplot(2,2,2)
        im = pcolormesh(freqs,hts,log10(ionograms[1,:,:]))
        im.set_clim(0.2,9.8)
        im.axes.set_xlim(2,12); im.axes.set_ylim(50,750); 
        im.axes.set_title(filename); im.axes.title.set_fontsize(10.0)
        colorbar()

        subplot(2,2,3)
        im = pcolormesh(freqs,hts,log10(ionograms[2,:,:]))
        im.set_clim(0.2,9.8)
        im.axes.set_xlim(2,12); im.axes.set_ylim(50,750); 
        im.axes.set_title(filename); im.axes.title.set_fontsize(10.0)
        colorbar()

        subplot(2,2,4)
        im = pcolormesh(freqs,hts,log10(ionograms[3,:,:]))
        im.set_clim(0.2,9.8)
        im.axes.set_xlim(2,12); im.axes.set_ylim(50,750); 
        im.axes.set_title(filename); im.axes.title.set_fontsize(10.0)
        colorbar()
        

        folderout='/raid/VALLEY+/VIPIR/processed/2015/%s' %folder_num
        fig.savefig(folderout+'/'+filename[6:6+13]+'.png')   # png format
        #show()   # display ionograms 
        #display.clear_output(wait=True)
        #display.display(gcf())
        fig.clf()
        
        whole_clean_procedure()
        pngno=pngno+1

/raid/VIPIR/vipir2015/014/raw/JM91J_2015014125004.RIQ
<open file '/raid/VIPIR/vipir2015/014/raw/JM91J_2015014125004.RIQ', mode 'rb' at 0x7f5836940780>
/raid/VIPIR/vipir2015/014/raw/JM91J_2015014125104.RIQ
<open file '/raid/VIPIR/vipir2015/014/raw/JM91J_2015014125104.RIQ', mode 'rb' at 0x7f58374e4f60>
/raid/VIPIR/vipir2015/014/raw/JM91J_2015014125204.RIQ
<open file '/raid/VIPIR/vipir2015/014/raw/JM91J_2015014125204.RIQ', mode 'rb' at 0x7f58368f6db0>
/raid/VIPIR/vipir2015/014/raw/JM91J_2015014125304.RIQ
<open file '/raid/VIPIR/vipir2015/014/raw/JM91J_2015014125304.RIQ', mode 'rb' at 0x7f5836940780>
/raid/VIPIR/vipir2015/014/raw/JM91J_2015014125404.RIQ
<open file '/raid/VIPIR/vipir2015/014/raw/JM91J_2015014125404.RIQ', mode 'rb' at 0x7f58374e48a0>
/raid/VIPIR/vipir2015/014/raw/JM91J_2015014125504.RIQ
<open file '/raid/VIPIR/vipir2015/014/raw/JM91J_2015014125504.RIQ', mode 'rb' at 0x7f58374e4f60>
/raid/VIPIR/vipir2015/014/raw/JM91J_2015014125604.RIQ
<open file '/raid/VIPIR/vipir2015/014/ra

/home/bye3/.local/lib/python2.7/site-packages/ipykernel_launcher.py:166: RuntimeWarning: divide by zero encountered in log10


/raid/VIPIR/vipir2015/014/raw/JM91J_2015014125804.RIQ
<open file '/raid/VIPIR/vipir2015/014/raw/JM91J_2015014125804.RIQ', mode 'rb' at 0x7f5836940780>
/raid/VIPIR/vipir2015/014/raw/JM91J_2015014125904.RIQ
<open file '/raid/VIPIR/vipir2015/014/raw/JM91J_2015014125904.RIQ', mode 'rb' at 0x7f58374e4f60>
/raid/VIPIR/vipir2015/014/raw/JM91J_2015014130004.RIQ
<open file '/raid/VIPIR/vipir2015/014/raw/JM91J_2015014130004.RIQ', mode 'rb' at 0x7f58374e48a0>
/raid/VIPIR/vipir2015/014/raw/JM91J_2015014130104.RIQ
<open file '/raid/VIPIR/vipir2015/014/raw/JM91J_2015014130104.RIQ', mode 'rb' at 0x7f58368f6db0>
/raid/VIPIR/vipir2015/014/raw/JM91J_2015014130204.RIQ
<open file '/raid/VIPIR/vipir2015/014/raw/JM91J_2015014130204.RIQ', mode 'rb' at 0x7f58374e48a0>
/raid/VIPIR/vipir2015/014/raw/JM91J_2015014130304.RIQ
<open file '/raid/VIPIR/vipir2015/014/raw/JM91J_2015014130304.RIQ', mode 'rb' at 0x7f58374e4f60>
/raid/VIPIR/vipir2015/014/raw/JM91J_2015014130404.RIQ
<open file '/raid/VIPIR/vipir2015/014/ra

/home/bye3/.local/lib/python2.7/site-packages/ipykernel_launcher.py:159: RuntimeWarning: divide by zero encountered in log10
/home/bye3/.local/lib/python2.7/site-packages/ipykernel_launcher.py:180: RuntimeWarning: divide by zero encountered in log10


/raid/VIPIR/vipir2015/014/raw/JM91J_2015014131504.RIQ
<open file '/raid/VIPIR/vipir2015/014/raw/JM91J_2015014131504.RIQ', mode 'rb' at 0x7f5836940780>
/raid/VIPIR/vipir2015/014/raw/JM91J_2015014131604.RIQ
<open file '/raid/VIPIR/vipir2015/014/raw/JM91J_2015014131604.RIQ', mode 'rb' at 0x7f58374e48a0>
/raid/VIPIR/vipir2015/014/raw/JM91J_2015014131704.RIQ
<open file '/raid/VIPIR/vipir2015/014/raw/JM91J_2015014131704.RIQ', mode 'rb' at 0x7f58374e4f60>
/raid/VIPIR/vipir2015/014/raw/JM91J_2015014131804.RIQ
<open file '/raid/VIPIR/vipir2015/014/raw/JM91J_2015014131804.RIQ', mode 'rb' at 0x7f58368f6db0>
/raid/VIPIR/vipir2015/014/raw/JM91J_2015014131904.RIQ
<open file '/raid/VIPIR/vipir2015/014/raw/JM91J_2015014131904.RIQ', mode 'rb' at 0x7f58374e4f60>
/raid/VIPIR/vipir2015/014/raw/JM91J_2015014132004.RIQ
<open file '/raid/VIPIR/vipir2015/014/raw/JM91J_2015014132004.RIQ', mode 'rb' at 0x7f58374e48a0>
/raid/VIPIR/vipir2015/014/raw/JM91J_2015014132104.RIQ
<open file '/raid/VIPIR/vipir2015/014/ra

...

<Figure size 1440x720 with 0 Axes>

<Figure size 1008x504 with 0 Axes>

<Figure size 1008x504 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 1008x504 with 0 Axes>

In [ ]:
# whole procedure, below this cell is individual step
iono_num = 0

mean_clear(vertical_noise_substraction(ionograms[iono_num,:,:]))
nnz = find_nnz(data_processed)
x_image = np.zeros(nnz)
y_image = np.zeros(nnz)
ind = 0
for i in range(data_processed.shape[0]):
    for j in range(data_processed.shape[1]):
        if data_processed[i, j] > 1:
            x_image[ind] = i;
            y_image[ind] = j;
            ind += 1

X_image = np.concatenate((array([y_image]), array([x_image])))
X_image = X_image.T

n_clusters = 75
plt.figure(figsize=(20, 10))
model = AgglomerativeClustering(linkage="single",
                                connectivity=None,
                                n_clusters=n_clusters)
model.fit(X_image)

group_want = find_cluster(X_image, model.labels_)
X_image_want = array(group_labels(X_image, model.labels_)[group_want])

x_group = X_image_want[:, 0]
y_group = X_image_want[:, 1]

data_final = np.zeros(data.shape)
for i in range(0, len(X_image_want)):
    row = int(X_image_want[i, 0]);
    col = int(X_image_want[i, 1]);
    data_final[col, row] = 10

figure(figsize=(14, 7))
im = pcolormesh(freqs,hts,data_final)
im.set_clim(0.2,9.8)
im.axes.set_xlim(2,12); im.axes.set_ylim(50,750); 
colorbar()
show(im)

In [ ]:
# computes new data array with vertical noise substraction
iono_num = 0

data = ionograms[iono_num,:,:]
data_after_substraction = vertical_noise_substraction(data)

plt.figure(figsize=(14, 7))
im = plt.pcolormesh(freqs,hts,np.log10(data_after_substraction))
im.set_clim(0.2,9.8)
im.axes.set_xlim(2,12); im.axes.set_ylim(50,750); 
plt.colorbar()
plt.show(im)

In [ ]:
# Power iteration to clear up
data_processed = mean_clear(data_after_substraction)

figure(figsize=(14, 7))
im = pcolormesh(freqs,hts,data_processed)
im.set_clim(0.2,9.8)
im.axes.set_xlim(2,12); im.axes.set_ylim(50,750); 
colorbar()
show(im)

In [ ]:
nnz = find_nnz(data_processed)
x_image = np.zeros(nnz)
y_image = np.zeros(nnz)
ind = 0
for i in range(data_processed.shape[0]):
    for j in range(data_processed.shape[1]):
        if data_processed[i, j] > 1:
            x_image[ind] = i;
            y_image[ind] = j;
            ind += 1

X_image = np.concatenate((array([y_image]), array([x_image])))
X_image = X_image.T
figure(figsize=(14, 7))
plt.scatter(X_image[:, 0], X_image[:, 1],marker = ".")

In [ ]:
n_clusters = 75
plt.figure(figsize=(20, 10))
model = AgglomerativeClustering(linkage="single",
                                connectivity=None,
                                n_clusters=n_clusters)
t0 = time.time()
model.fit(X_image)
plt.scatter(X_image[:, 0], X_image[:, 1], c=model.labels_,
             cmap=plt.cm.nipy_spectral, marker = ".")
plt.title('linkage=%s\n(time %.2fs)' % ("single", elapsed_time),
          fontdict=dict(verticalalignment='top'))

plt.show()

In [ ]:
group_want = find_cluster(X_image, model.labels_)
X_image_want = array(group_labels(X_image, model.labels_)[group_want])

x_group = X_image_want[:, 0]
y_group = X_image_want[:, 1]

figure(figsize = (14,7))
plt.scatter(x_group, y_group, marker = ".")

In [ ]:
data_final = np.zeros(data.shape)
for i in range(0, len(X_image_want)):
    row = int(X_image_want[i, 0]);
    col = int(X_image_want[i, 1]);
    data_final[col, row] = 10

figure(figsize = (14, 7))
im = pcolormesh(freqs,hts,log10(data))
im.set_clim(0.2,9.8)
im.axes.set_xlim(2,12); im.axes.set_ylim(50,750); 
colorbar()

figure(figsize=(14, 7))
im = pcolormesh(freqs,hts,data_final)
im.set_clim(0.2,9.8)
im.axes.set_xlim(2,12); im.axes.set_ylim(50,750); 
colorbar()
show(im)